In [ ]:
!pip install openai

In [ ]:
import openai
import pandas as pd
import time

openai.api_key = 'api_key'

df = pd.read_excel('cleaned_truncated_context.xlsx')

def generate_qa_from_context(context):
    response = openai.chat.completions.create(model="gpt-4o",
     messages=[
        {"role": "system", "content": "You are a knowledgeable assistant capable of generating questions and answers based on provided context. \
                                       Here are some examples:\
                                       \n\nContext: The ACB's pre-tax profit increased by more than VND15,024 billion in the first nine months of 2023, an increase of 11% over the same period of the previous year, despite a strong increase in reserve extraction. Overall, in the first nine months of 2023, ACB's principal income increased by 9% over the same period of the previous year, with a net improvement in retail revenues of nearly VND18,670 billion.\
                                       \nQuestion: What was the percentage increase in ACB's pre-tax profit in the first nine months of 2023?\
                                       \nAnswer:11% over the same period the previous year.\
                                       \n\nGiven the context:"},
        {"role": "user", "content": f"{context}\n\nGenerate a relevant question and answer, question is simple and answer is short:"}
    ])
    output = response.choices[0].message.content.strip()

    question_start = output.find("Question:") + len("Question:")
    answer_start = output.find("Answer:") + len("Answer:")
    if question_start > -1 and answer_start > -1:
        question = output[question_start:answer_start - len("Answer:")].strip()
        answer = output[answer_start:].strip()
    else:

        question = output
        answer = ""
    return question, answer

def save_progress(index):
    with open('progress.txt', 'w') as file:
        file.write(str(index))

def load_progress():
    try:
        with open('progress.txt', 'r') as file:
            progress_str = file.read().strip()
            if progress_str:
                return int(progress_str)
            else:
                return 0
    except FileNotFoundError:
        return 0

last_index = load_progress()
qa_count = 0

for index, row in df.iterrows():
    if index < last_index:
        continue
    if pd.isna(row['question']) or pd.isna(row['answer']):
        question, answer = generate_qa_from_context(row['context'])
        df.at[index, 'question'] = question
        df.at[index, 'answer'] = answer
        qa_count += 1
        save_progress(index)
        df.to_excel('result.xlsx', index=False)

        if qa_count % 3 == 0:
           time.sleep(60)